# Count No. of people and Density Estimation

### Workflow
1. Open the video file using cv2.VideoCapture to start frame reading.
2. Read frames sequentially inside a loop until the video ends.
3. Run YOLO detection and tracking on each frame for the PERSON class only
4. Extract bounding boxes and unique tracking IDs for detected people.
5. Compute centroid coordinates for each detected bounding box.
6. Draw bounding boxes, centroids, and IDs for visual debugging.
7. Calculate the number of people currently visible in the frame.
8. Classify congestion level based on live people thresholds.
11. Overlay analytics (count, density, line) onto the frame.
12. Display the processed frame for monitoring.



In [1]:
!pip install ultralytics opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s eta 0:00:00


In [2]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from google.colab import files
import numpy as np


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print("Video:", video_path)

Saving retail_checkout_counter.mp4 to retail_checkout_counter (1).mp4
Video: retail_checkout_counter (1).mp4


In [7]:
model = YOLO("yolov8s.pt")


In [ ]:
# Open the uploaded video file
cap = cv2.VideoCapture(video_path)

# Virtual horizontal line position (adjust based on your video)
LINE_Y = 300

# Set to store IDs that have already been counted
counted_ids = set()

# Final people crossing counter
counter = 0

# Process video frame-by-frame
while True:

    # Read one frame from the video
    ret, frame = cap.read()

    # If no frame is returned → video ended
    if not ret:
        break

    # Run YOLO detection + tracking on the frame
    # persist=True keeps track IDs across frames
    # classes=[0] means detect PERSON class only
    results = model.track(frame, persist=True, classes=[0])

    # Variable to store number of people currently visible
    live_people = 0

    # Check if tracker produced valid IDs
    if results[0].boxes.id is not None:

        # Extract bounding box coordinates (x1, y1, x2, y2)
        boxes = results[0].boxes.xyxy.cpu().numpy()

        # Extract unique tracking IDs for each detected person
        ids = results[0].boxes.id.cpu().numpy()

        # Number of people currently in frame
        live_people = len(boxes)

        # Loop through every detected person
        for box, obj_id in zip(boxes, ids):

            # Convert box coordinates to integers
            x1, y1, x2, y2 = map(int, box)

            # Compute centroid (center of bounding box)
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # Draw bounding box around person
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

            # Draw centroid point
            cv2.circle(frame, (cx, cy), 5, (0,0,255), -1)

            # Display tracking ID above box
            cv2.putText(frame, f"ID {int(obj_id)}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

            # ---------------- COUNTING LOGIC ----------------

            # Check if this ID was never counted before
            if obj_id not in counted_ids:

                # If centroid crosses the counting line
                if cy >= LINE_Y:

                    # Increase counter
                    counter += 1

                    # Mark this ID as counted
                    counted_ids.add(obj_id)

    # ---------------- DENSITY ESTIMATION ----------------

    # Classify congestion level based on live people count
    if live_people <= 5:
        density = "Low Density"
    elif live_people <= 15:
        density = "Medium Density"
    elif live_people <= 30:
        density = "High Density"
    else:
        density = "Highly Congested"

    # Draw horizontal counting line
    cv2.line(frame, (0, LINE_Y), (frame.shape[1], LINE_Y), (255,0,0), 2)

    # Display total crossing count
    cv2.putText(frame, f"Count: {counter}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

    # Display number of people currently visible
    cv2.putText(frame, f"Live People: {live_people}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 3)

    # Display congestion level
    cv2.putText(frame, density, (20, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)

    # Show frame inside Colab
    cv2_imshow(frame)

# Release video resource
cap.release()

# Close any OpenCV windows
cv2.destroyAllWindows()


# Queue Length Estimation (Cashier Queue)

### Workflow

1. Open the video file using cv2.VideoCapture for frame extraction.
2. Define a rectangular queue zone using fixed coordinates.
3. Read frames sequentially inside a loop until the video ends.
4. Run YOLO detection on each frame for the PERSON class only.
5. Initialize a queue counter for the current frame.
6. Extract bounding box coordinates for detected persons.
7. Compute centroid coordinates for each bounding box.
8. Check whether each centroid lies inside the queue zone.
9. Increase queue length if the centroid falls within the zone.
10. Assign red color for queued persons and green for others.
11. Draw bounding boxes and centroid markers for visualization.
12. Draw the queue zone rectangle on the frame.
13. Overlay the queue length text onto the frame.
14. Display frames at intervals to maintain performance.


In [3]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print("Video:", video_path)

Saving retail_queue_estimation.mp4 to retail_queue_estimation.mp4
Video: retail_queue_estimation.mp4


In [19]:
DISPLAY_EVERY = 5      # ✅ define ONCE
max_frames = 300       # ✅ safety stop (very important)
frame_count = 0        # ✅ define ONCE

In [ ]:
cap = cv2.VideoCapture(video_path)

# Queue area (adjust for your video)
QUEUE_X1, QUEUE_Y1 = 600, 400
QUEUE_X2, QUEUE_Y2 = 950, 980

while True:

    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Stop condition (prevents endless run)
    if frame_count > max_frames:
        break

    results = model(frame, classes=[0])  # detect persons only

    queue_count = 0

    if len(results[0].boxes) > 0:
        boxes = results[0].boxes.xyxy.cpu().numpy()

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)

            # Compute centroid
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # Check if inside queue zone
            if QUEUE_X1 <= cx <= QUEUE_X2 and QUEUE_Y1 <= cy <= QUEUE_Y2:
                queue_count += 1
                color = (0, 0, 255)  # red → in queue
            else:
                color = (0, 255, 0)  # green → normal

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 255), -1)

    # Draw queue zone
    cv2.rectangle(frame, (QUEUE_X1, QUEUE_Y1), (QUEUE_X2, QUEUE_Y2), (255,0,0), 2)

    # Display queue length
    cv2.putText(frame, f"Queue Length: {queue_count}", (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

    # Show only some frames (keeps Colab fast)
    if frame_count % DISPLAY_EVERY == 0:
        cv2_imshow(frame)

cap.release()
cv2.destroyAllWindows()
